# extract-pages-from-mongo-v7
SanjayKAroraPhD@gmail.com <br>
December 2018

## Description
This version of the notebook extracts groups of pages from mongodb by firm_name to create firm-centric <b>about</b> page output files that can later be topic modeled.  In doing so, it removes repetitive content (e.g., repeated menu items) and garbage content (e.g., improperly parsed HTML code). 

## Change log
v4 focuses on about pages

## TODO:
* Whole process: get data, topic model and see if it looks sufficiently interesting/different
* Enhance data collection, per the following: 
    * Select a region or country — WAIT 
        * http://www.ivoclarvivadent.com: Please select your region
        * https://www.enersys.com/: PLEASE SELECT A REGION
        * https://www.m-petfilm.com/: ENGLISH
    * Crawl from focal about page only following links that look like part of the about story, maintaining ordering.  Check to see if the other links identified above are also there? 
        * http://xtalsolar.com/investors_partners.html
* Order known about us pages in the same way the links are found on a home page or about us landing page

In [2]:
# import data processing and other libraries
import csv
import sys
import requests
import os
import re
import pprint
import pymongo
import traceback
from time import sleep
import requests
import pandas as pd
import io
from IPython.display import display
import time
import numpy as np
from bs4 import BeautifulSoup
import string
import random
from urllib.parse import urlparse, urljoin
from collections import defaultdict
from collections import OrderedDict
import collections

In [3]:
from boilerpipe.extract import Extractor

ModuleNotFoundError: No module named 'boilerpipe'

In [3]:
# import sklearn
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [4]:
MONGODB_DB = "FirmDB_20181226"
MONGODB_COLLECTION = "pages_ABOUT2"
CONNECTION_STRING = "mongodb://localhost"

client = pymongo.MongoClient(CONNECTION_STRING)
db = client[MONGODB_DB]
col = db[MONGODB_COLLECTION]

ABOUT_DIR = '/Users/sarora/dev/EAGER/data/orgs/about/'
DATA_DIR = '/Users/sarora/dev/EAGER/data/orgs/parsed_page_output/'
TRAINING_PERCENT = .10

PHRASE_LENGTH = 60
MIN_PARA_LEN = 5

pp = pprint.PrettyPrinter()

In [5]:
# output urls for labeling of training data
results = col.find({},{"url": 1, "firm_name": 1})
df = pd.DataFrame(columns = ('firm_name', 'url'))
for i in range(results.count()):
    result = results.next()
    url = result['url'][0]
    firm_name = result['firm_name'][0] if 'firm_name' in result else ''
    df.loc[i] = [firm_name, url]
    
df['gid'] = df.groupby(['firm_name']).ngroup()

In [6]:
df.gid.nunique()
label_ids = random.sample(range(1, df.gid.nunique()), 200)
df_label = df[df['gid'].isin(label_ids)]
with open(ABOUT_DIR + 'about_pages_to_label.csv', mode='w') as to_label:
    df_label.to_csv(to_label, index=False)

In [7]:
# read back labeled data (note that about, management/team and partners, are dichotomous)
df_about_labeled = pd.read_csv(ABOUT_DIR + 'about_pages_labeled_v4.csv')
df_about_labeled = df_about_labeled.fillna(0)
df_about_labeled['pages_in_domain_ftr'] = df_about_labeled.groupby(["firm_name"])["url"].transform("count")

labeled_urls = list(df_about_labeled['url']) # for training models on labeled urls below
df_about_labeled = df_about_labeled.set_index(['firm_name', 'url'])
df_about_labeled.head()

# final test set is the rows of the original data frame without the urls in df_about_labeled 

about_lbl  \
firm_name                        url                                                             
3M Innovative Properties Company https://www.3m.com/                                       0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...        0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/          1.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...        0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...        1.0   

                                                                                     mgmt_lbl  \
firm_name                        url                                                            
3M Innovative Properties Company https://www.3m.com/                                      0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...       0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/         0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...       0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...       0.0   

                                                                                     partners_lbl  \
firm_name                        url                                                                
3M Innovative Properties Company https://www.3m.com/                                          0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...           0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/             0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...           0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...           0.0   

                                                                                     ip_lbl  \
firm_name                        url                                                          
3M Innovative Properties Company https://www.3m.com/                                    0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...     0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/       0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...     0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...     0.0   

                                                                                     about_agg_lbl  \
firm_name                        url                                                                 
3M Innovative Properties Company https://www.3m.com/                                           0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...            0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/              1.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...            0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...            0.0   

                                                                                     gid  \
firm_name                        url                                                       
3M Innovative Properties Company https://www.3m.com/                                 1.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/    1.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about

In [8]:
df_about_labeled.shape

(1455, 7)

## Create features to predict about pages
Create features:
1. number of domain pages (identified above)
2. whether a given page is an about us page (as opposed to a home page)
3. number of words on a page (and share of words on a page given all candidate pages)
4. number of sentences (and share fo sentences on a page given all candidate pages)
5. title and url path fragment unigrams (also tried n-grams) with worse results
6. descriptor text around the focal link (as identified upstream when crawling as as persisted to mongodb then)

Other text-based ideas for features may be found here: https://towardsdatascience.com/understanding-feature-engineering-part-3-traditional-methods-for-text-data-f6f7d70acd41

In [9]:
# pattern regex to remove unwanted words that show up in topic models
p = re.compile(r"(\(\)|''|``|\"|null|ul|li|ol|^\.|^:|^/|\\|--|cooki|'s|corpor|busi|inc\.|ltd|co\.|compan|keyboard|product|technolog)", flags=re.IGNORECASE)

# remove html content
def is_javascript (x):
    match_string = r"(CDATA|return\s+true|return\s+false|getelementbyid|function|\w+\(.*?\);|\w{2,}[\\.|:]+\w{2,}|header|hover|'\w+':\s+'\w+|\\|{|}|\r|\n|\/\/')"
    # capture CDATA; function declarations; function calls; word sequences separated by a period (e.g., denoting paths)
    regex = re.findall(match_string, x) 
    # check to see if the regex finds some percentage of the words look like javascript patterns
    if (len(regex) / float(len(x.split())) > .10):
        return True 
    else:
        return False

def clean_page_content (text_list):
    # remove whatever we think is html
    removed_html = filter(lambda x: not( bool(BeautifulSoup(x, "html.parser").find()) ), text_list)
    # remove content that looks like javascript 
    removed_js = filter(lambda x: not (is_javascript(x)), removed_html)
    # add other checks here as needed

    return removed_js

# this method called from below
def count_page_features (result): 
    if not result:
        return 0, 0
    
    # get number of words
    running_text = ''
    clnd_text = clean_page_content(result['full_text'])
    clnd_text = '\n'.join(clnd_text)
    boilerpipe = None

    if 'body' in result:
        extractor = Extractor(extractor='DefaultExtractor', html = result['body'][0])
        lines = extractor.getText().replace(u'\xa0', u' ').split('\n')
        filtered = filter(lambda x: not re.match(r'^\s*$', x), lines)
        boilerpipe = '\n'.join(filtered)

    # TODO fix to split().  Counting characters currently 
    if boilerpipe and (len(boilerpipe) > .5 * len(clnd_text)):
        running_text += boilerpipe
    else:
        running_text += clnd_text
    
    num_words = len(running_text.split())
    num_sentences = 0
    
    # loop over text and add title elements to the paragraph they describe
    document = running_text.split('\n')
    for i in range(len(document)): # figure out a way to chunk groups of content
        if len(document[i]) <= 12 or len(document[i].split()) < MIN_PARA_LEN : # maybe a menu or simple pagragraph heading? 
            continue
        num_sentences += len(re.findall(r'(\.|;|\!)( |$)', document[i])) # count what appears to be number of sentences sentence

    # pp.pprint (joined)
    return num_words, num_sentences

In [13]:
# remove simple article words and punctuation (need to keep 'about')
stop_words = ['the','a'] + list(string.punctuation) 
# remove known company names for model training and evaluation in the labeled data 
remove_regex = re.compile(r'^(3m|united|states|menu|en_us|algeternal|s\d+|sarepta|skygen|nexgen|abbott|adlens|errorpage|\d{1,3}|\d{5,}|\w+\d+|\d+\w+|asten|johnson|baker|hughes|ge|bhge|biocon|egfr|gcsf|biocon|pegfilgrastim|bostik|canon|chevron|phillips|coloplast|cyberonics|microsoft|evoqua|ford|hitachi|glucanbio|hunter|douglas|kimberly|clark|lextar|fisher|lockheed|martin |lux|nec|nanocopoeia|cisco|schlumberger|weccamerica|inanobio|nanocomposix|zoetis|zygo)$', re.IGNORECASE)
# used to filter top-level header content
header_in = re.compile('(about|company|corporate|who.we.are|(^|/)vision|awards|profile|corporate|management|team|history|values|strategy|our |technology|research|commercialization)', flags=re.IGNORECASE)
header_regex = re.compile(r'h[1-9]+')

def get_domain (url):
    o = urlparse(url.lower())
    domain = o.netloc.strip('www.')
    return domain

def strip_firm_name (firm_name, text):
    strip_regex = re.compile(r"(" + "\s|".join(firm_name.split()) + "\s)", re.IGNORECASE)
    clnd_text = strip_regex.sub ('', text)
    
    more_regex = re.compile(r"([A-Z]\.?){1,} ")
    clnd_text = more_regex.sub ('', clnd_text)
    
    return ' '.join(clnd_text.split(' '))

# standard firm cleaning regex
def clean_firm_name (firm):
    firm_clnd = re.sub('(\.|,| corporation| incorporated| llc| inc| international| gmbh| ltd)', '', firm, flags=re.IGNORECASE).rstrip()
    return firm_clnd

def clean_string(in_string):
    if not in_string:
        return in_string
    split_words = in_string.lower().split()
    result_words  = [word for word in split_words if word not in stop_words]
    result_words  = [word for word in result_words if not remove_regex.search(word)]
    result = ' '.join(result_words)
    return ' ' + result

def get_page_path_text (url):
    o = urlparse(url.lower())
    path = o.path
    path_parts = path.split ('/')
    path_parts = [part.split('.')[0] for part in path_parts] # remove page names
    path_parts = [split for part in path_parts for split in part.split('-') ] # split on underscores, hyphens, et al
    path_parts = [split for part in path_parts for split in part.split('_') ] 
    clnd_string = clean_string(' '.join(path_parts))
    return clnd_string

# recurse through the header text to add into feature grams
def get_header_text (headers, names, index):
    texts = [clean_string(header.text) for header in headers if header.name == names[index]]
    texts = list(filter(header_in.search, texts))
    if texts and len(texts[0].split()) > 4:
        if(len(names) > (index + 1)):
            return get_header_text (headers, names, index + 1)
        else:
            return ''
    else: 
        return ' '.join (texts)

# load page data and create features (this method kicks everything else off)
def process_firms (urls): 
    firm_text_features = {}
    firm_count_features = {} # ['is_about', 'num_words', 'num_sentences']
    
    for url in urls: 
        result = col.find_one({"url": url})
        if not result:
            result = col.find_one({"orig_url": url})
        if not result: # just can't find the page
            print ('Cannot find ' + url)
            continue
        
        # --------------------
        # text based features
        # --------------------
        firm_name = result
        domain = get_domain(url)
        
        if 'html' not in result:
            print ('Cannot find html for', url)
            continue
            
        html = result['html'][0]
        
        descriptor_text = ''
        
        # text from the text wrapping the link
        descriptor = result['descriptor'][0].replace ('|', '').replace('None', '')
        if descriptor: 
            descriptor_text = clean_string(descriptor)
        
        # path text within the url 
        path_text = get_page_path_text(url)
         
        # title from the html page
        soup = BeautifulSoup(html, 'lxml')
        title_text = ''
        if soup.title and soup.title.string:
            # print (soup.title.string)
            title_text = clean_string(soup.title.string)
        
        # headers from the page
        header_text = ''
        headers = soup.find_all(header_regex, text=True)
        names = sorted(set ([header.name for header in headers]))
        header_text = get_header_text (headers, names, 0)

        firm_name = result['firm_name'][0]
        firm_text_features[url] = [strip_firm_name(firm_name, descriptor_text), strip_firm_name(firm_name, path_text), strip_firm_name(firm_name, title_text), strip_firm_name(firm_name, header_text)]
        
        # --------------------
        # count based features
        # --------------------        
        is_about = int(result['is_about'][0])
        num_words, num_sentences = count_page_features (result)
        firm_count_features[url] = [is_about, num_words, num_sentences]
        
    return firm_text_features, firm_count_features

In [14]:
# Test various methods and regex used above 
print (get_page_path_text ('http://biocon.com/biocon_aboutus_businesses.asp'))

print (get_page_path_text('http://www.google.com/path-en/path_to/page.html'))
print (re.split("\W+|_", "Testing this_thing"))
print (clean_string('3m 01	08	100	10m ford 235 1990 s129 188209 0913lk the ? about us'))
print (clean_string('3m 01	08	100	10m ford 235 1990 s129 188209 0913lk the ? about us'))
pp.pprint (list(filter(header_in.search, ['about us', 'not found', 'company'])))

print (strip_firm_name('Ford Motor Company', 'This Ford Motor Company has been around for a while.'))
print (strip_firm_name (clean_firm_name('Ford Motor Company'), 'Ford is a motor company.  It has been building vehicles for over a century. H.W.F_ Ford was a nice guy.'))

 aboutus businesses
 path en path to page
['Testing', 'this', 'thing']
 about us
 about us
['about us', 'company']
This has been around for a while.
is a company.  It has been building vehicles for over a century. H.W.F_ was a nice guy.


In [15]:
# get firm website data for n-gram processing AND grab count features
labeled_firm_text_features, labeled_firm_count_features = process_firms (labeled_urls)

Cannot find http://www.genomichealth.com/en-US/


In [16]:
# testing, should be  
# pages about us about us – about us 
# [1, 882, 19]
print(labeled_firm_text_features['https://nanocomposix.com/pages/about-us'])
print(labeled_firm_count_features['https://nanocomposix.com/pages/about-us'])

['', ' pages about us', ' about us –', ' about us']
[1, 882, 19]


### Process firm text features into a vectorized format

In [17]:
urls = labeled_firm_text_features.keys() # create in an order
print (len(urls))
corpus = []


for url in urls:
    running_text = ''
    descriptor_text = labeled_firm_text_features[url][0]
    path_text = labeled_firm_text_features[url][1]
    title_text = labeled_firm_text_features[url][2]
    header_text = labeled_firm_text_features[url][3]
    running_text = descriptor_text + path_text + title_text + header_text
    corpus.append (running_text)
    
# unigram
ubv = TfidfVectorizer(min_df=0., max_df=1.)
# you can set the n-gram range to 1,2 to get unigrams as well as bigrams (performs worse than just unigrams)
# ubv = TfidfVectorizer(ngram_range=(1,2)) 

ubv_matrix = ubv.fit_transform(corpus)

ubv_matrix = ubv_matrix.toarray()
vocab = ubv.get_feature_names()
ubv_df = pd.DataFrame(ubv_matrix, columns=vocab)
ubv_df.index = urls
ubv_df.index.name='url'
ubv_df.head()

1454


,003,10,10m,10mseriesafinancing,13485,14001,1933,1961,1962,1975,...,z18038e,zegage,zeitung,zero,zerowastemanagement,zoetis,zonne,中文,公司简介,隆達電子
url,,,,,,,,,,,,,,,,,,,,,
https://www.3m.com/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.3m.com/3M/en_US/company-us/3m-science-applied-to-life/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.3m.com/3M/en_US/company-us/about-3m/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.3m.com/3M/en_US/company-us/about-3m/state-of-science-index-survey/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.3m.com/3M/en_US/company-us/about-3m/technologies/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
count_df = pd.DataFrame.from_dict(labeled_firm_count_features, orient='index', columns = ['is_about_ftr', 'num_words_ftr', 'num_sentences_ftr'])
count_df.index.name = 'url'
count_df.head()

,is_about_ftr,num_words_ftr,num_sentences_ftr
url,,,
https://www.3m.com/,1,3487,210
https://www.3m.com/3M/en_US/company-us/3m-science-applied-to-life/,1,1140,14
https://www.3m.com/3M/en_US/company-us/about-3m/,1,854,38
https://www.3m.com/3M/en_US/company-us/about-3m/state-of-science-index-survey/,1,1033,9
https://www.3m.com/3M/en_US/company-us/about-3m/technologies/,1,3739,175


In [19]:
# processed above but output here for clarity
df_about_labeled.head()

about_lbl  \
firm_name                        url                                                             
3M Innovative Properties Company https://www.3m.com/                                       0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...        0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/          1.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...        0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...        1.0   

                                                                                     mgmt_lbl  \
firm_name                        url                                                            
3M Innovative Properties Company https://www.3m.com/                                      0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...       0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/         0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...       0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...       0.0   

                                                                                     partners_lbl  \
firm_name                        url                                                                
3M Innovative Properties Company https://www.3m.com/                                          0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...           0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/             0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...           0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...           0.0   

                                                                                     ip_lbl  \
firm_name                        url                                                          
3M Innovative Properties Company https://www.3m.com/                                    0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...     0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/       0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...     0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...     0.0   

                                                                                     about_agg_lbl  \
firm_name                        url                                                                 
3M Innovative Properties Company https://www.3m.com/                                           0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...            0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/              1.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...            0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...            0.0   

                                                                                     gid  \
firm_name                        url                                                       
3M Innovative Properties Company https://www.3m.com/                                 1.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/    1.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about

### Merge text-based features with count features and labels for model training and evaluation

In [20]:
# merge datasets (features and labeled data)
print(ubv_df.shape)
print(df_about_labeled.shape)
print(count_df.shape)

merged = ubv_df.join(df_about_labeled, how='inner')

labeled = merged.join(count_df, how='inner')
labeled['num_words_firm_ftr'] = labeled['num_words_ftr'].groupby(level=0).transform('sum')
labeled['share_of_words_ftr'] = labeled['num_words_ftr'] / labeled['num_words_firm_ftr']

labeled['num_sentences_firm_ftr'] = labeled['num_sentences_ftr'].groupby(level=0).transform('sum')
labeled['share_of_sentences_ftr'] = labeled['num_sentences_ftr'] / labeled['num_sentences_firm_ftr']

print(labeled.shape)
labeled.head()

(1454, 2416)
(1455, 7)
(1454, 3)
(1454, 2430)


003  \
firm_name                        url                                                       
3M Innovative Properties Company https://www.3m.com/                                 0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   

                                                                                      10  \
firm_name                        url                                                       
3M Innovative Properties Company https://www.3m.com/                                 0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   

                                                                                     10m  \
firm_name                        url                                                       
3M Innovative Properties Company https://www.3m.com/                                 0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   

                                                                                     10mseriesafinancing  \
firm_name                        url                                                                       
3M Innovative Properties Company https://www.3m.com/                                                 0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...                  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/                    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...                  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...                  0.0   

                                                                                     13485  \
firm_name                        url                                                         
3M Innovative Properties Company https://www.3m.com/                                   0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/      0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...    0.0   

                                                                                     14001  \
firm_name                        url                                                         
3M Innovative Properties Company https://www.3m.com/                                   0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/      0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...    0.0   

                                                                           

In [21]:
# check for missing urls in the final labeled data frame
for url in labeled_urls: # labeled data
    if url not in list(labeled.index.levels[1]): # feature data
        print ('Missing', url)

labeled.xs('https://nanocomposix.com/pages/about-us', level=1)

Missing http://www.genomichealth.com/en-US/


,003,10,10m,10mseriesafinancing,13485,14001,1933,1961,1962,1975,...,about_agg_lbl,gid,pages_in_domain_ftr,is_about_ftr,num_words_ftr,num_sentences_ftr,num_words_firm_ftr,share_of_words_ftr,num_sentences_firm_ftr,share_of_sentences_ftr
firm_name,,,,,,,,,,,,,,,,,,,,,
nanoComposix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1049.0,3,1,882,19,3116,0.283055,102,0.186275


In [22]:
# labeled train/test split
print (len(ubv_df.columns))
X = labeled.iloc[:,1:len(ubv_df.columns)]
       
# other in other non text-based features
# 1. number of domain pages (identified above)
# 2. is home page and doesn't have any other pages (is_sole_page)
# 3. whether a given page is an about us page (as opposed to a home page)
# 4. number of words on a page (and share of words)
# 5. number of sentences (and share of sentences)

X['pages_in_domain_ftr'] = np.reciprocal(labeled['pages_in_domain_ftr']) # in
X['is_about_ftr'] = labeled['is_about_ftr'] # in
# X['num_words_ftr'] = labeled['num_words_ftr']
# X['share_of_words_ftr'] = labeled['share_of_words_ftr']
# X['num_sentences_ftr'] = labeled['num_sentences_ftr']
X['share_of_sentences_ftr'] = labeled['share_of_sentences_ftr'] # in
X.to_csv(ABOUT_DIR + 'X.csv', index = True) # for manual inspection

# normalize
# X = (X - X.mean()) / X.std()
# X = (X - X.min()) / (X.max() - X.min())

y = labeled.loc[:,'about_lbl']
print (X.shape)
print (y.shape)
X.head()

2416
(1454, 2418)
(1454,)


10  \
firm_name                        url                                                       
3M Innovative Properties Company https://www.3m.com/                                 0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   

                                                                                     10m  \
firm_name                        url                                                       
3M Innovative Properties Company https://www.3m.com/                                 0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...  0.0   

                                                                                     10mseriesafinancing  \
firm_name                        url                                                                       
3M Innovative Properties Company https://www.3m.com/                                                 0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...                  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/                    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...                  0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...                  0.0   

                                                                                     13485  \
firm_name                        url                                                         
3M Innovative Properties Company https://www.3m.com/                                   0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/      0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...    0.0   

                                                                                     14001  \
firm_name                        url                                                         
3M Innovative Properties Company https://www.3m.com/                                   0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/      0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...    0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...    0.0   

                                                                                     1933  \
firm_name                        url                                                        
3M Innovative Properties Company https://www.3m.com/                                  0.0   
                                 https://www.3m.com/3M/en_US/company-us/3m-scien...   0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m/     0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...   0.0   
                                 https://www.3m.com/3M/en_US/company-us/about-3m...   0.0   

                                                                     

## Train and evaluate the model
On just the labeled data

In [23]:
# specify a few models

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "SVC", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    SVC(gamma=0.001, C=100.), 
    QuadraticDiscriminantAnalysis()]

In [24]:
# build dataframe for output metrics 
eval_df = pd.DataFrame (names,index=(range(len(names))), columns=["Name"])
eval_df['Accuracy'] = np.float64(0)

In [25]:
# build evaluation outputs (currently limited to accuracy)
i = np.int64(0)
for name, clf in zip(names, classifiers):
    display (name)
    scores = cross_val_score(clf, X, y)
    avg_score = np.mean(scores)
    eval_df.set_value(i, 'Accuracy', avg_score)
    i = i + 1
    
display(eval_df)
eval_df.to_clipboard()
# Neural net work best

'Nearest Neighbors'

'Linear SVM'

'RBF SVM'

'Decision Tree'

'Random Forest'

...

'AdaBoost'

'Naive Bayes'

'SVC'

'QDA'

,Name,Accuracy
0,Nearest Neighbors,0.825310
1,Linear SVM,0.732463
2,RBF SVM,0.843191
3,Decision Tree,0.828068
4,Random Forest,0.732463
5,Neural Net,0.889282
6,AdaBoost,0.864521
7,Naive Bayes,0.732427
8,SVC,0.848021
9,QDA,0.453269


## Grid search using MLPClassifier to tune hyperparameters
The above results clearly show that a type of feed-forward neural network is the most accurate type of model

In [26]:
hls = []
# hls.append([20,])
# hls.append([70,])
hls.append([100,])
# hls.append([50,50])
# hls.append([70,70,70])
# hls.append([40,40,40])
# hls.append([10,10,10])
# hls.append([50,50,50,50])
pp.pprint(hls)

[[100]]


In [27]:
parameters = {'solver': ['adam'], 'max_iter': [50, 100], 'alpha': 10.0 ** -np.arange(1, 3), 'hidden_layer_sizes': hls}
clf_grid = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
clf_grid.fit(X,y)

print("Best score: %0.4f" % clf_grid.best_score_)
print("Using the following parameters:")
print(clf_grid.best_params_)

Best score: 0.8927
Using the following parameters:
{'alpha': 0.1, 'hidden_layer_sizes': [100], 'max_iter': 50, 'solver': 'adam'}


In [28]:
# train neural net model with best hyperparameter configuration
clf = MLPClassifier(alpha=0.1, hidden_layer_sizes=(100,), max_iter=100, solver='adam')
clf.fit(X, y)

y_hat = clf.predict(X)
print(confusion_matrix(y, y_hat))

[[1061    4]
 [   9  380]]


In [29]:
# print all instances where predictions don't match labels (for inspection)
for key, y_i, y_hat_i in zip(list(X.index), y, y_hat):
    if y_i != y_hat_i:
        print(key[1], 'has been classified as ', y_hat_i, 'but should be ', y_i) 

https://www.agfa.com/corporate/investor-relations/corporate-governance/ has been classified as  0.0 but should be  1.0
https://www.cadence.com/content/cadence-www/global/en_US/home/company/corporate-governance.html has been classified as  0.0 but should be  1.0
https://www.cookmedical.com/about/ethics-compliance/ has been classified as  0.0 but should be  1.0
http://www.deltaww.com/about/brand.aspx?secID=5&pid=2&tid=0&hl=en-US has been classified as  1.0 but should be  0.0
http://ecovativedesign.com/ has been classified as  0.0 but should be  1.0
https://www.eni.com/en_IT/innovation.page has been classified as  0.0 but should be  1.0
http://hiqsolar.com/contact-us.html has been classified as  0.0 but should be  1.0
https://www.kashivpharma.com/about/people-management/ has been classified as  0.0 but should be  1.0
https://nanocopoeia.com/ has been classified as  1.0 but should be  0.0
https://www.nanosysinc.com/who-we-are/ has been classified as  0.0 but should be  1.0
http://www.olymp

## Predict about pages for unlabeled data

In [30]:
# prepare domain level features 
df_predict = df[~df['url'].isin(labeled_urls)] 
df_predict['pages_in_domain_ftr'] = df_predict.groupby(["firm_name"])["url"].transform("count")

df_predict = df_predict.set_index(['firm_name', 'url'])
df_predict = df_predict.fillna(0)

df_predict = df_predict.sort_values(by=['gid'])
print (df_predict.shape)
df_predict.head()

(8208, 2)


gid  \
firm_name            url                                               
22nd Century Limited http://www.xxiicentury.com/history/           0   
                     http://www.xxiicentury.com/our-management/    0   
                     http://www.xxiicentury.com/                   0   
                     http://www.xxiicentury.com/profile/           0   
ABB AB               https://new.abb.com/investorrelations         2   

                                                                 pages_in_domain_ftr  
firm_name            url                                                              
22nd Century Limited http://www.xxiicentury.com/history/                           4  
                     http://www.xxiicentury.com/our-management/                    4  
                     http://www.xxiicentury.com/                                   4  
                     http://www.xxiicentury.com/profile/                           4  
ABB AB               https://new.abb.com/investorrelations                        17

In [31]:
# check to see whether there are duplicate urls
# note: there should be because different assignees may map to the same domain (see error above)
counter=collections.Counter(df_predict.index)
most_common = counter.most_common(5)
pp.pprint (most_common)

[(('22nd Century Limited', 'http://www.xxiicentury.com/history/'), 1),
 (('22nd Century Limited', 'http://www.xxiicentury.com/our-management/'), 1),
 (('22nd Century Limited', 'http://www.xxiicentury.com/'), 1),
 (('22nd Century Limited', 'http://www.xxiicentury.com/profile/'), 1),
 (('ABB AB', 'https://new.abb.com/investorrelations'), 1)]


In [32]:
# prepare n-gram and count features (takes a few minutes to run)
unlabeled_firm_text_features, unlabeled_firm_count_features = process_firms (set(df_predict.index.get_level_values('url')))

Cannot find html for https://www.bridgestone.com/corporate/news/rss/bridgestone_global_news.xml
Cannot find html for https://www.fujielectric.com/company/news/rss/rss.xml


In [33]:
unlabeled_firm_count_features['http://www.foxconn.com/Investors_En/Corporate_Governance.html?index=2']

[1, 56, 0]

In [34]:
prediction_urls = unlabeled_firm_text_features.keys()
pred_corpus = []

for url in prediction_urls:
    running_text = ''
    descriptor_text = unlabeled_firm_text_features[url][0]
    path_text = unlabeled_firm_text_features[url][1]
    title_text = unlabeled_firm_text_features[url][2]
    header_text = unlabeled_firm_text_features[url][3]
    running_text = descriptor_text + path_text + title_text + header_text
    pred_corpus.append (running_text)

ubv_prediction_matrix = ubv.transform(pred_corpus)

ubv_prediction_matrix = ubv_prediction_matrix.toarray()
vocab = ubv.get_feature_names()
ubv_prediction_df = pd.DataFrame(ubv_prediction_matrix, columns=vocab)
ubv_prediction_df.index = prediction_urls
ubv_prediction_df.index.name='url'
ubv_prediction_df.head()

,003,10,10m,10mseriesafinancing,13485,14001,1933,1961,1962,1975,...,z18038e,zegage,zeitung,zero,zerowastemanagement,zoetis,zonne,中文,公司简介,隆達電子
url,,,,,,,,,,,,,,,,,,,,,
https://www.riverbed.com/partners/index.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.eisai.com/index.html?redirect_ir=management/index.html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://corporate.exxonmobil.com/en/community/womens-economic-opportunity/weoi-partners/overview,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
https://www.ibm.com/us-en/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
http://sensor-kinesis.com/our-team/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
count_pred_df = pd.DataFrame.from_dict(unlabeled_firm_count_features, orient='index', columns = ['is_about_ftr', 'num_words_ftr', 'num_sentences_ftr'])
count_pred_df.index.name = 'url'
print (count_pred_df.shape)
count_pred_df.head()

(8206, 3)


,is_about_ftr,num_words_ftr,num_sentences_ftr
url,,,
https://www.riverbed.com/partners/index.html,1,671,13
https://www.eisai.com/index.html?redirect_ir=management/index.html,1,606,4
https://corporate.exxonmobil.com/en/community/womens-economic-opportunity/weoi-partners/overview,1,2529,28
https://www.ibm.com/us-en/,0,800,12
http://sensor-kinesis.com/our-team/,1,387,11


In [36]:
# merge datasets (features and labeled data)
print(ubv_prediction_df.shape)
print(df_predict.shape)
print(count_pred_df.shape)

predict_merged = ubv_prediction_df.join(df_predict, how='inner')

unlabeled = predict_merged.join(count_pred_df, how='inner')
unlabeled['num_sentences_firm_ftr'] = unlabeled['num_sentences_ftr'].groupby(level=0).transform('sum')
unlabeled['share_of_sentences_ftr'] = unlabeled['num_sentences_ftr'] / unlabeled['num_sentences_firm_ftr']
# fill in empties with 0's
unlabeled.loc[np.isnan(unlabeled['share_of_sentences_ftr']), 'share_of_sentences_ftr'] = 0 

print(unlabeled.shape)
unlabeled.head()

(8206, 2416)
(8208, 2)
(8206, 3)
(8206, 2423)


003   10  \
firm_name            url                                                    
22nd Century Limited http://www.xxiicentury.com/history/         0.0  0.0   
                     http://www.xxiicentury.com/our-management/  0.0  0.0   
                     http://www.xxiicentury.com/                 0.0  0.0   
                     http://www.xxiicentury.com/profile/         0.0  0.0   
ABB AB               https://new.abb.com/investorrelations       0.0  0.0   

                                                                 10m  \
firm_name            url                                               
22nd Century Limited http://www.xxiicentury.com/history/         0.0   
                     http://www.xxiicentury.com/our-management/  0.0   
                     http://www.xxiicentury.com/                 0.0   
                     http://www.xxiicentury.com/profile/         0.0   
ABB AB               https://new.abb.com/investorrelations       0.0   

                                                                 10mseriesafinancing  \
firm_name            url                                                               
22nd Century Limited http://www.xxiicentury.com/history/                         0.0   
                     http://www.xxiicentury.com/our-management/                  0.0   
                     http://www.xxiicentury.com/                                 0.0   
                     http://www.xxiicentury.com/profile/                         0.0   
ABB AB               https://new.abb.com/investorrelations                       0.0   

                                                                 13485  14001  \
firm_name            url                                                        
22nd Century Limited http://www.xxiicentury.com/history/           0.0    0.0   
                     http://www.xxiicentury.com/our-management/    0.0    0.0   
                     http://www.xxiicentury.com/                   0.0    0.0   
                     http://www.xxiicentury.com/profile/           0.0    0.0   
ABB AB               https://new.abb.com/investorrelations         0.0    0.0   

                                                                 1933  1961  \
firm_name            url                                                      
22nd Century Limited http://www.xxiicentury.com/history/          0.0   0.0   
                     http://www.xxiicentury.com/our-management/   0.0   0.0   
                     http://www.xxiicentury.com/                  0.0   0.0   
                     http://www.xxiicentury.com/profile/          0.0   0.0   
ABB AB               https://new.abb.com/investorrelations        0.0   0.0   

                                                                 1962  1975  \
firm_name            url                                                      
22nd Century Limited http://www.xxiicentury.com/history/          0.0   0.0   
                     http://www.xxiicentury.com/our-management/   0.0   0.0   
                     http://www.xxiicentury.com/                  0.0   0.0   
                     http://www.xxiicentury.com/profile/          0.0   0.0   
ABB AB               https://new.abb.com/investorrelations        0.0   0.0   

                                                                          ...            \
firm_name            url                                                  ...             
22nd Century Limited http://www.xxiicentury.com/history/                  ...             
                     http://www.xxiicentury.com/our-management/           ...             
                     http://www.xxiicentury.com/                          ...             
                     http://www.xxiicentury.com/profile/                  ...             
ABB AB               https://new.abb.com/investorrelations                ...             

                                                                  中文  公司简介  \
firm_name  

In [37]:
# merge
X_test = unlabeled.iloc[:,1:len(ubv_prediction_df.columns)]

print (X.shape)
print (X_test.shape) # should be the same number of cols

X_test['pages_in_domain_ftr'] = np.reciprocal(unlabeled['pages_in_domain_ftr'])
X_test['is_about_ftr'] = unlabeled['is_about_ftr']
X_test['share_of_sentences_ftr'] = unlabeled['share_of_sentences_ftr']
X_test.to_csv(ABOUT_DIR + 'X_test.csv', index = True) # for manual inspection

(1454, 2418)
(8206, 2415)


In [38]:
# predict with newly constructed X
y_predicted = clf.predict(X_test)

In [39]:
y_hat = y_predicted.astype(int) 
print (type(y_hat))

<class 'numpy.ndarray'>


In [40]:
# write to file
def find_home_urls (firm_names):
    urls = [] # return object 
    for fn in firm_names: 
        result = col.find_one({"firm_name": fn, "is_about": "0"}, {'orig_url': 1})
        if not result: 
            print ("Cannot find home page result for", fn)
        else: 
            urls.append(result['orig_url'][0])
    return urls
    

about_pages = []
has_about_firm_names = {}

# output predicted values to file
for fn, u, predicted_value in zip(X_test.index.get_level_values('firm_name'), X_test.index.get_level_values('url'), y_predicted):
    # print (cfn + ' with url ' + u + ' has predicted value ' + str(predicted_value))
    about_pages.append([fn, u, predicted_value, 0])
    if predicted_value == 1.: 
        has_about_firm_names[fn] = 1
        
# and the labeled ones too...
for fn, u, labeled_value in zip(X.index.get_level_values('firm_name'), X.index.get_level_values('url'), y):
    # print (cfn + ' with url ' + u + ' has predicted value ' + str(labeled_value))
    about_pages.append([fn, u, labeled_value, 0])
    if labeled_value == 1: 
        has_about_firm_names[fn] = 1
    
about_df = pd.DataFrame (about_pages, columns = ('firm_name', 'url', 'is_about', 'default_to_home'))
about_df.set_index(['firm_name', 'url'], inplace=True)

firm_names = list(about_df.index.levels[0])
no_positive_pred_firm_names = set(firm_names).difference(set(has_about_firm_names)) # len is 97
default_home_urls = find_home_urls (no_positive_pred_firm_names)
# print (default_home_urls)

Cannot find home page result for Nanocopocia
Cannot find home page result for Schneider Electric Solar Inverters USA
Cannot find home page result for Community Power Corporation
Cannot find home page result for AGC Flat Glass North America
Cannot find home page result for Coactive Drive Corporation
Cannot find home page result for Gilead Sciences
Cannot find home page result for AT&T Corporation
Cannot find home page result for eNow
Cannot find home page result for Seagate Technology LLC
Cannot find home page result for Biological Dynamics


In [41]:
# check output for firm or url
about_df.iloc[about_df.index.get_level_values('firm_name') == "Cima NanoTech Israel Ltd"]
# about_df.iloc[about_df.index.get_level_values('url') == "https://spc-intl.com/(X(1)S(zopdv2d2knciwj2xbkkpfqfz))/?AspxAutoDetectCookieSupport=1"]

,,is_about,default_to_home
firm_name,url,,


In [42]:
# in the data frame set these pages to is_about = 1 and default_to_home = 1
about_df.iloc[about_df.index.get_level_values('url').isin (default_home_urls), 0:2] = 1
about_df.iloc[about_df.index.get_level_values('url').isin (default_home_urls), 0:2].head()

,,is_about,default_to_home
firm_name,url,,
AC International Inc.,http://acinter.com/,1.0,1
ALSTOM Technology Ltd,https://www.alstom.com/,1.0,1
ASTUTE MEDICAL,http://www.astutemedical.com/,1.0,1
Advanced Silicon Group,http://www.advancedsilicongroup.com/,1.0,1
Aerogen,https://www.aerogen.com/,1.0,1


In [43]:
# clean firm names
about_out_df = about_df.reset_index()
about_out_df['firm_name'] = about_out_df['firm_name'].apply(lambda x: clean_firm_name(x))
about_out_df.head()

# write df to csv
about_out_df.to_csv(ABOUT_DIR + 'about_predicted_and_labels.csv', index = False)

## Extract data from mongodb
* Now that we know which pages are about pages, extract from mongodb and output for topic modeling
* For now, construct paragraphs from different pages by ordering urls by their length.  In the future, might want to contruct paragraphs in their 'natural' sequential order as they would appear on a home page or landing page

In [44]:
# combine both labeled and predicted frames
# TODO: add in homepages where needed! -- ska (2/19/19)
print (X_test.shape)
print(X.shape)

combined = X_test.append(X)
print (combined.shape)
print (len(y_predicted))
print (len(y))
abouts = pd.DataFrame(index=combined.index)

abouts['is_about'] = list(y_predicted) + list(y)
abouts = abouts.reset_index()
abouts.head()

(8206, 2418)
(1454, 2418)
(9660, 2418)
8206
1454


,firm_name,url,is_about
0,22nd Century Limited,http://www.xxiicentury.com/history/,1.0
1,22nd Century Limited,http://www.xxiicentury.com/our-management/,0.0
2,22nd Century Limited,http://www.xxiicentury.com/,1.0
3,22nd Century Limited,http://www.xxiicentury.com/profile/,1.0
4,ABB AB,https://new.abb.com/investorrelations,0.0


In [45]:
# gather unique firm_names from mongodb
firm_names = set(abouts['firm_name'])
print (len(firm_names))
pp = pprint.PrettyPrinter()
pp.pprint(firm_names)

1146
{'22nd Century Limited',
 '3M Innovative Properties Company',
 'ABB AB',
 'AC International Inc.',
 'ACACIA RESEARCH GROUP LLC',
 'ACUCELA INC.',
 'ACell',
 'ADASA INC.',
 'ADMA Products',
 'ADVANCED CERAMIC FIBERS',
 'ADVANCED INNOVATION CENTER LLC',
 'AFMODEL',
 'AG ENERGY SOLUTIONS',
 'AGC Flat Glass North America',
 'AGFA-GEVAERT N.V.',
 'AIR PRODUCTS AND CHEMICALS',
 'ALGETERNAL TECHNOLOGIES',
 'ALSTOM Technology Ltd',
 'AMPT',
 'APPLIED STEMCELL',
 'ARBOR THERAPEUTICS',
 'AROG PHARMACEUTICALS',
 'ASCENT SOLAR TECHNOLOGIES',
 'ASM America',
 'ASML Netherlands B.V.',
 'ASTUTE MEDICAL',
 'AT&T Corporation',
 'ATC Technologies',
 'ATOMERA INCORPORATED',
 'ATTOSTAT',
 'AVI BioPharma',
 'AVOGY',
 'AbbVie Inc.',
 'Abbott Cardiovascular Systems Inc.',
 'Abbott Molecular Inc.',
 'Abbott Point of Care Inc.',
 'Abengoa Bioenergy New Technologies',
 'Ablexis',
 'Access Business Group International LLC',
 'AccuRay Corporation',
 'Accuri Cytometers',
 'Acer Incorporated',
 'Achushnet Comp

In [46]:
def get_ordered_about_urls (firm_name):
    print (firm_name)
    urls = list (abouts.loc[(abouts['firm_name'] == firm_name), 'url'])
    urls.sort(key = len)
    print ('Original urls')
    pp.pprint(urls)

    index = {}
    for url in urls:
        path_fragments = len(url.split('/'))
        added = False
        for i in range(1, path_fragments):
            key_phrase = url.rsplit('/', maxsplit=i)[0]
            if key_phrase in urls or (key_phrase + '/') in urls: 
                od = index.setdefault(key_phrase, OrderedDict())
                od[url] = 1
                added = True
                continue
        if not added:
            od = index.setdefault(url, OrderedDict())
            od[url] = 1
 
    # pp.pprint (index)
    
    return_urls = []
    seen = set ()
    for key in index.keys():
        tree_urls = index[key]
        for fu in tree_urls:
            if fu not in seen:
                return_urls.append(fu)
                seen.add(fu)
    
    # finally remove home page if it exists and if there are other pages to draw on
    if not return_urls: 
        return None
    else: 
#         first_page = return_urls[0]
#         first_page_path = get_page_path_text (first_page)
#         if first_page_path == ' ' or first_page_path == '':
#             print (first_page_path + 'empty')
#             return_urls.pop(0)
        return return_urls

test_urls = get_ordered_about_urls ('Previvo Genetics')
print ('Ordered urls')
pp.pprint (test_urls)

Previvo Genetics
Original urls
['https://previvo.com/']
Ordered urls
['https://previvo.com/']


In [47]:
# iterate through firm urls and return concatenated string
def get_content (urls): 
    running_text = ''
    for url in urls:
        print ('\tWorking on ' + url)
        result = col.find_one( {"url": url} )
        if result:
            clnd_text = clean_page_content(result['full_text'])
            clnd_text = '\n'.join(clnd_text)
            boilerpipe = None
            
            if 'body' in result:
                extractor = Extractor(extractor='DefaultExtractor', html = result['body'][0])
                lines = extractor.getText().replace(u'\xa0', u' ').split('\n')
                filtered = filter(lambda x: not re.match(r'^\s*$', x), lines)
                boilerpipe = '\n'.join(filtered)

            # TODO fix to split().  Counting characters currently 
            if boilerpipe and (len(boilerpipe) > .5 * len(clnd_text)):
                print ('\t\tUsing boilerplate')
                running_text += boilerpipe
            else:
                print ('\t\tUsing clnd_text')
                running_text += clnd_text
        else:
            print ('Cannot find url: ' + url)

    return running_text

In [48]:
# regex test 
regex = re.findall(r"(CDATA|return\s+true|return\s+false|getelementbyid|function|\w+\(.*?\);|\w{2,}[\\.|:]+\w{2,}|'\w+':\s+'\w+|\\|{|}|\r|\n|\/\/')", 
                   "CDATA function contact-us getelementbyid javascript.function linker:autoLink www.littlekidsinc.com fxnCall(param.param); email@dextr.us 'type': 'image' return true return false rev7bynlh\\u00252bvcgrjg\\ {height}") # last part is words sequences separated by punct

In [49]:
print (test_urls)
test_site_text = get_content (test_urls)
print (test_site_text)

['https://previvo.com/']
	Working on https://previvo.com/
		Using boilerplate
Begin typing your search above and press return to search.
Learn More
Empowering Women & Couples with Fertility Choices
Previvo offers a new choice, a new medical device, and a new procedure that empowers women and couples, both fertile and infertile, in assisted reproduction. Our proprietary and patented medical device is designed with you and your potential baby in mind.
Our studies demonstrate that we can conduct Previvo Uterine Lavage while maintaining safety of the woman and the embryo(s).
The Previvo Uterine Lavage System is built upon robust and extensive intellectual property and clinical testing. We continue to invest in our intellectual portfolio by filing new methods and system improvements. With over 130 lavages, The Previvo Uterine Lavage System has demonstrated the ability to recover healthy embryos.
The Previvo Uterine Lavage System empower women and couples with a new assisted reproductive tec

In [50]:
# run process_firm and write to file
pp = pprint.PrettyPrinter()
for firm_name in firm_names: 
    print ("Working on " + firm_name)
    about_urls = get_ordered_about_urls(firm_name)
    if not about_urls:
        print ("\tCouldn't find any urls for firm!")
        firm_urls = df_predict.xs(firm_name, level=0)
        home_page = firm_urls.loc[firm_urls['is_about_ftr'] == 0]
        about_urls.append(home_page.index.get_level_values('url'))
        
    about_text = get_content (about_urls)
    
    if about_text: 
        firm_clnd = clean_firm_name(firm_name) # standard cleaning code throughout project
        about_clnd = strip_firm_name (firm_name, about_text)
        file = re.sub('\/', '|', firm_clnd) + '.txt'
        with io.open(DATA_DIR + file,'w',encoding='utf8') as f:
            f.write (about_clnd)
    else:
        print ("\tCouldn't find any text for firm!")

Working on Mitsubishi Electric Corporation
Mitsubishi Electric Corporation
Original urls
['https://www.mitsubishielectric.com/en/index.html',
 'https://www.mitsubishielectric.com/en/about/index.html',
 'https://www.mitsubishielectric.com/en/about/rd/index.html',
 'https://www.mitsubishielectric.com/en/about/rd/ip/index.html',
 'https://www.mitsubishielectric.com/en/about/mission/index.html',
 'https://www.mitsubishielectric.com/en/about/history/index.html',
 'https://www.mitsubishielectric.com/en/about/message/index.html',
 'https://www.mitsubishielectric.com/en/about/strategy/index.html',
 'https://www.mitsubishielectric.com/en/about/locations/index.html',
 'https://www.mitsubishielectric.com/en/about/rd/maisart/index.html',
 'https://www.mitsubishielectric.com/en/about/rd/advance/index.html',
 'https://www.mitsubishielectric.com/en/about/businesses/index.html',
 'https://www.mitsubishielectric.com/en/about/procurement/index.html',
 'https://www.mitsubishielectric.com/en/about/history

		Using clnd_text
	Working on https://www.mitsubishielectric.com/en/about/procurement/green/accreditation/index.html
		Using clnd_text
	Working on https://www.mitsubishielectric.com/en/about/businesses/transportation_systems/index.html
		Using clnd_text
	Working on https://www.mitsubishielectric.com/en/about/businesses/airconditioning_refrigeration_systems/index.html
		Using clnd_text
Working on Roche Molecular Systems
Roche Molecular Systems
Original urls
['https://www.roche.com/about.htm',
 'https://www.roche.com/about/ceo.htm',
 'https://www.roche.com/partnering.htm',
 'https://www.roche.com/about/people.htm',
 'https://www.roche.com/about/history.htm',
 'https://www.roche.com/about/business.htm',
 'https://www.roche.com/about/chairman.htm',
 'https://www.roche.com/about/priorities.htm',
 'https://www.roche.com/about/governance.htm',
 'https://www.roche.com/about/our_purpose.htm',
 'https://www.roche.com/about/our-strategy.htm',
 'https://www.roche.com/about/people/leaders.htm',
 'h

		Using clnd_text
	Working on https://www.roche.com/about/business/diagnostics.htm
		Using clnd_text
	Working on https://www.roche.com/about/governance/committees.htm
		Using boilerplate
	Working on https://www.roche.com/about/business/disease_areas.htm
		Using clnd_text
	Working on https://www.roche.com/about/business/pharmaceuticals.htm
		Using clnd_text
	Working on https://www.roche.com/about/business/roche_worldwide.htm
		Using clnd_text
	Working on https://plus.google.com/browser-not-supported/?ref=/share
		Using boilerplate
	Working on https://www.roche.com/about/governance/code_of_conduct.htm
		Using boilerplate
	Working on https://www.roche.com/about/governance/board_of_directors.htm
		Using clnd_text
	Working on https://www.roche.com/partnering/pharma-areas-of-interest.htm
		Using clnd_text
	Working on https://www.roche.com/about/governance/executive_committee.htm
		Using clnd_text
	Working on https://www.roche.com/sustainability/approach/risk-management.htm
		Using boilerplat

	Working on http://gojo.com/Corporate-Featured-News
		Using clnd_text
	Working on https://career4.successfactors.com/career?company=gojo
		Using clnd_text
	Working on http://gojo.com/en/Markets/Offices-Workplace/Office-Building-Management
		Using boilerplate
	Working on http://gojo.com/en/Corporate-Hand-Hygiene-Hub/Hand-Hygiene-Tools-And-Downloads
		Using clnd_text
Working on Dow Global Technologies Inc.
Dow Global Technologies Inc.
Original urls
['https://www.dow.com/en-us', 'https://corporate.dow.com/en-us/about/locations']
	Working on https://www.dow.com/en-us
		Using clnd_text
	Working on https://corporate.dow.com/en-us/about/locations
		Using clnd_text
Working on Kodak Alaris Inc.
Kodak Alaris Inc.
Original urls
['https://www.kodakalaris.com/',
 'https://www.kodakalaris.com/company',
 'https://www.kodakalaris.com/company/careers',
 'https://www.kodakalaris.com/company/locations',
 'https://www.kodakalaris.com/company/contact-us',
 'https://www.kodakalaris.com/company/leadership',


		Using boilerplate
	Working on https://www.hollingsworth-vose.com/en/Company/Management-Bios/Jochem-Hofstetter/
		Using clnd_text
	Working on https://www.hollingsworth-vose.com/en/Company/Management-Bios/David-von-Loesecke/
		Using clnd_text
Working on Lux Bio Group
Lux Bio Group
Original urls
['http://luxbiogroup.com/', 'http://luxbiogroup.com/about.html']
	Working on http://luxbiogroup.com/
		Using clnd_text
	Working on http://luxbiogroup.com/about.html
		Using boilerplate
Working on Tessera
Tessera
Original urls
['https://www.tessera.com/']
	Working on https://www.tessera.com/
		Using clnd_text
Working on BRIGHTLEAF TECHNOLOGIES INC.
BRIGHTLEAF TECHNOLOGIES INC.
Original urls
['http://www.brightleafpower.com/',
 'http://www.brightleafpower.com/values/',
 'http://www.brightleafpower.com/about-us/',
 'http://www.brightleafpower.com/innovation/',
 'http://www.brightleafpower.com/executive-leadership/']
	Working on http://www.brightleafpower.com/
		Using boilerplate
	Working on http://

...

		Using clnd_text
Working on Dialogic Corporation
Dialogic Corporation
Original urls
['https://www.dialogic.com/',
 'https://www.dialogic.com/company',
 'https://www.dialogic.com/about-us',
 'https://www.dialogic.com/partners',
 'https://www.dialogic.com/partners/ibm',
 'https://www.dialogic.com/company/awards',
 'https://www.dialogic.com/company/cookies',
 'https://www.dialogic.com/company/privacy',
 'https://www.dialogic.com/company/quality',
 'https://www.dialogic.com/partners/directory',
 'https://www.dialogic.com/profile/login-help',
 'https://www.dialogic.com/company/socialmedia',
 'https://www.dialogic.com/company/terms-of-use',
 'https://www.dialogic.com/company?sc_lang=fr-fr',
 'https://www.dialogic.com/company/executive-team',
 'https://www.dialogic.com/company/online-press-kit',
 'https://www.dialogic.com/partners/avaya-devconnect',
 'https://www.dialogic.com/company/board-of-directors',
 'https://www.dialogic.com/company/environmental-policy',
 'https://www.dialogic.com/pro

		Using boilerplate
	Working on http://www.jfe-steel.co.jp/en/company/steel.html
		Using clnd_text
	Working on http://www.jfe-steel.co.jp/en/company/about.html
		Using clnd_text
	Working on http://www.jfe-steel.co.jp/en/company/index.html
		Using clnd_text
	Working on http://www.jfe-steel.co.jp/en/company/philosophy.html
		Using boilerplate
	Working on http://www.jfe-steel.co.jp/en/company/facilities.html
		Using boilerplate
	Working on http://www.jfe-steel.co.jp/en/company/management.html
		Using clnd_text
Working on MEDport
MEDport
Original urls
['https://medport.com/']
	Working on https://medport.com/
		Using boilerplate
Working on Meso Scale Technologies
Meso Scale Technologies
Original urls
['https://www.mesoscale.com/',
 'https://www.mesoscale.com/en/our_company',
 'https://www.mesoscale.com/en/our_company/news',
 'https://www.mesoscale.com/en/our_company/careers',
 'https://www.mesoscale.com/en/our_company/contact_us',
 'https://www.mesoscale.com/en/our_company/your-feedback',
 

		Using clnd_text
	Working on https://www.hamamatsu.com/jp/en/our-company/history/index.html
		Using boilerplate
	Working on https://www.hamamatsu.com/jp/en/our-company/about-crl/index.html
		Using clnd_text
	Working on https://www.hamamatsu.com/jp/en/our-company/at-a-glance/index.html
		Using boilerplate
	Working on https://www.hamamatsu.com/jp/en/our-company/our-philosophy/index.html
		Using boilerplate
	Working on https://www.hamamatsu.com/jp/en/our-company/business-domain/index.html
		Using clnd_text
	Working on https://www.hamamatsu.com/jp/en/our-company/corporate-profile/index.html
		Using boilerplate
	Working on https://www.hamamatsu.com/jp/en/our-company/investor-relations/index.html
		Using clnd_text
	Working on https://www.hamamatsu.com/jp/en/our-company/presidents-message/index.html
		Using boilerplate
	Working on https://www.hamamatsu.com/jp/en/our-company/csr/quality-control/index.html
		Using boilerplate
	Working on https://www.hamamatsu.com/jp/en/our-company/global-organ

		Using clnd_text
	Working on http://www.nmm.jx-group.co.jp/english/company/
		Using clnd_text
	Working on http://www.nmm.jx-group.co.jp/english/sustainability/
		Using clnd_text
	Working on http://www.nmm.jx-group.co.jp/english/company/data.html
		Using boilerplate
	Working on http://www.nmm.jx-group.co.jp/english/company/chart.html
		Using clnd_text
	Working on http://www.nmm.jx-group.co.jp/english/company/history.html
		Using boilerplate
	Working on http://www.nmm.jx-group.co.jp/english/company/message.html
		Using boilerplate
	Working on http://www.nmm.jx-group.co.jp/english/company/investor.html
		Using clnd_text
	Working on http://www.nmm.jx-group.co.jp/english/company/philosophy.html
		Using clnd_text
	Working on http://www.nmm.jx-group.co.jp/english/sustainability/index.html
		Using clnd_text
	Working on http://www.nmm.jx-group.co.jp/english/company/group/group_01.html
		Using boilerplate
	Working on http://www.nmm.jx-group.co.jp/english/company/japan/outline_j.html
		Using cln

		Using clnd_text
	Working on http://www.bemis.com/about-bemis
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/about-us
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/citizenship
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/sustainability
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/bemis-graphics
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/corporate-governance
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/about-us/our-leadership
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/bemis-innovation-center
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/about-us/our-work-matters
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/citizenship/bemis-company-foundation
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/sustainability/social-sustainability
		Using clnd_text
	Working on http://www.bemis.com/about-bemis/sustainability/economic-